## word2vec 속도 개선
- CBOW의 문제점은 corpus에 포함된 어휘 수가 많아지면 계산량도 커진다는 점입니다.
- 이번 장의 목표는 word2vec의 속도 개선이다.
- 첫번째 개념으로 Embedding이라는 새로운 계층의 도입이다.
- 두번째 개념으로 Negative Sampling이라는 새로운 손실함수를 도입한다.
- 두가지 개선을 통해 진짜 word2vec을 완성할 수 있다.

## 4.1 word2vec 개선
- CBOW 모델은 단어 2개를 맥락으로 사용해, 이를 바탕으로 하나의 단어(타겟)을 추측한다.
- 만약 어휘가 100만 개일 떄를 가정하여 CBOW 모델을 살펴보면 문제점을 알 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/765abe6a-3b2d-452d-b90a-44b1690f0abf)

- 2가지 연산에 대해 병목이 나타난다.
- 입력층의 원핫 표현과 가중치 행렬 Win의 곱 계산 (4.1 절에서 해결)
- 은닉층과 가중치 행렬 Wout의 곱 및 소프트맥스 계층 연산 (Negative Sampling을 도입하여 4.2 절에서 해결)

#### 4.1.1 Embedding 계층
- word2vec 구현에서는 단어를 원핫 표현으로 바꿧다.
- 입력 (원핫 벡터)와 가중치를 행렬 연산한다.
- 그런데 이떄 원핫 벡터와 가중치를 행렬 연산하면 원핫 벡터의 1에 대응 하는 열에 해당하는 weight 행을 추출하는 것 뿐이다.
- 이 행위를 할때 행렬 연산 자체는 연산량만 많이 들 뿐이다.
- 따라서 단어 ID에 해당하는 Weight 행을 추출하는 계층을 따로 만드는 것이 더 효율적이다.
- 이 계층을 Embedding 계층이라 한다.

#### 4.1.2 Embedding 계층 구현
- 행렬에서 특정 행을 추출하는 것은 쉽다.
- Numpy에서 index를 사용하여 특정 행을 추출할 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/d382c1a1-d4ed-450c-9af7-0800a2efaa95)

- idx가 여러개 일떄 dh가 같은 index에 덮여지는 것을 주의 해야 한다.
- 따라서 동일한 idx에 값이 있을 땐 덮어 쓰는 것이 아닌 더하기를 해야한다.
- Embedding 계층의 구현을 통해서 MatMul 계층의 연산 복잡도를 생략할 수 있게 됬다.


In [2]:
import numpy as np
W = np.arange(21).reshape(7, 3)

# numpy array를 이용하여 지정된 행을 추출할 수 있다.
idx = np.array([1, 0, 3, 0])
print(W[idx]) 

class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]

        return out

    def backward(self, dout):
        '''
        - Embedding 계층의 순전파는 가중치 W의 특정행을 추출할 뿐이다.
        - 단순히 가중치의 특정행 뉴런만을 다음 층으로 흘려 보내는 것이다.
        - 따라서 역전파에서는 앞 층으로부터 
        '''
        dW, = self.grads
        dW[...] = 0

        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]

        return None

    

[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


## word2vec 개선 2
- 남은 병목은 은닉층 이후의softmax 계층의 계산이다.
- Negative Sampling 기법을 이용한다.
- Negative Sampling은 어휘가 아무리 많아져도 계산량을 낮은 수준에서 일정하게 억제할 수 있다.

#### 4.2.1 은닉칭 이후 계산의 문제점
- 은닉층 이후 계싼의 문제점을 알아보기 위해, 은닉층 뉴련이 100개일 떄 word2vec(CBOW 모델)을 예를 들어 본다.
- 은닉층 이후 계산이 오래 걸리는 부분은 두 부분이다.
- 은닉층의 뉴런과 가중치 행렬(Wout)의 곱
- Softmax 계층의 계산

#### 4.2.2 다중 분류에서 이진 분류로
- Negative Sampling의 핵심 아이디어는 '이진 분류'에 있다, 정확하게 말하면 '다중 분류'를 '이진 분류'로 근사하는 것이 네거티브 샘플링을 이해 하는 중요한 포인트 이다.
- 맥락이 주어졌을때 어떤 타겟이 나올까라는 문제에서 맥락이 주어졌을때 이것이 타겟입니까? 라고 문제를 바꾼다.

![image](https://github.com/choibigo/Study/assets/38881179/8a118dc2-c3da-4389-a18b-7762ed14228e)

- 이떄 1개의 출력에 sigmoid를 취해 그 단어가 타겟일 확률을 구한다.

#### 4.2.3 시그모이드 함수와 교차 엔트로피 오차
- 이진문제로 풀기 위해 점수에 시그모이드 함수를 적용해 확률로 변환하고, 손실 함수를 구할 때는 Binary Cross Entropy를 사용한다.

![image](https://github.com/choibigo/Study/assets/38881179/2eaa8a2d-363e-4c1e-9aaf-5e3e8350e222)

![image](https://github.com/choibigo/Study/assets/38881179/37f99b25-64f4-4264-85d8-b76cc0a8592c)

- y^은 sigmoid의 출력이고 y는 정답 레이블이다.
- 이때 주목할 점은 역전파는 (예측값 - 정답값)이다. 즉, 두 값의 차이이다.
- 정답이 1이라면 예측값이 1에 가까울 수록 오차가 0에 가까워진다.
- 따라서, 오차가 크면 크게 학습하고 오차가 작으면 작게 학습하게 된다.

#### 4.2.4 다중 분류에서 이진 분류로 구현
- 다중 분류에서는 출력층의 노드 수는 어휘수와 동일 했다. 따라서 100만개의 분류를 위해서는 출력층에 100만개의 노드수가 필요했다.
- 그러나 이진 분류에서는 은닉층의 output과 타겟의 벡터 내적연산만을 통해서 타겟인지 아닌지에 대한 이진 확률을 출력한다.
- 추가적으로 target에 대한 내적 연산만 수행하기 때문에 Embedding 계층(특정 행렬만 가져오는 연산)이후 내적 연산을해 연산량을 줄일 수 있다.

![image](https://github.com/choibigo/Study/assets/38881179/19038d3d-51c7-4770-9611-17c2f722a04d)


In [1]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx) # embedding의 forward는 연산할 특정 행(weight값) 을 가져오는 것이다. 
        out = np.sum(target_W*h, axis=1) # 원소곱 이후에 더한다 = 내적연산
        
        self.cache = (h, target_W) # cache에 hidden layer와 target weight를 담아 둔다 => backward에서 사용
        return out

    def backward(self, dout):
        h, target_W = self.cache

        dout = dout.reshape(dout.shape[0], 1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

#### 4.2.5 네거티브 샘플링
- 지금까지는 긍정적인 예(정답)에 대해서만 학습했다.
- 정답 이외의 단어에 대해서는 어떠한 학습도 하지 못했다.
- 정답에 대해서는 sigmoid 계층의 출력을 1에 가깝게 만들고, 부정적인 예는 sigmoid 계층의 출력을 0에 가깝게 만드는 것이 필요하다.
- 만약 맥락이 "you"와 "goodbye"일 때 타겟이 "hello"일 확률은 낮은 값이여야 바람직 하다.
- 긍정적인 예와 부정적인 예 모두를 대상으로 문제를 생각해야한다.
- 부정적인 예를 모두 대상으로 학습하면 어떨까? => 기존에 사용한 softmax와 다를게 없어진다.
- 근본적인 해법으로 부정적인 예를 N개 선택해야한다.
- 적은 수의 부정적 예를 샘풀링해서 사용한다, 이것이 '네거티브 샘플링' 기법이다.

![image](https://github.com/choibigo/Study/assets/38881179/15e04906-fbbd-4cf3-b6cc-13ac416a239c)

- 정답에 대해서는 정답 레이블을 1을 주고 샘플링된 오답에 대해서는 정답 레이블을 0으로 준다.

#### 4.2.6 네거티브 샘플링의 샘플링 기법
- 추가적으로 샘플링을 어떻게 할것인지도 중요하다.
- 무작위로 샘플링 하는 것보다 좋은 기법이있다.
- corpus의 통계 데이터를 기초로 샘풀링 하는 것이다.
- corpus에 자주 등장하는 많이 추출하고 드믈게 등장하는 단어는 적게 추출하는 것이다.
- corpus에서의 단어 빈도를 기준으로 샘플링하며면, 각 단어의 출현 횟수를 구해 '확률 분포'로 나타낸다.
- 그런 다음 그 확률분포대로 단어를 샘플링 하면 된다.

![image](https://github.com/choibigo/Study/assets/38881179/248e71ef-4b50-4ce1-8f95-887aa3c9c3a9)

- 흔한 단어를 잘 선택하는것이 모델의 성능을 좌우한다.
- 원래 확률 분포의 각 요소에 0.75제곱을 해준다.
- 그이유는 출현 확률이 낮은 단어를 완전히 배제하지 않기 위해서 이다.
- 정확히 말하면 0.75 제곱을 함으로써 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있다.

```py
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p) # [0.64196878 0.33150408 0.02652714]
```
- 낮은 확률의 단어가 좀금 더 쉽게 샘플링 되도록 하기 위한 구제 조치를 하면서 완전히 배제 하지 못하게 한다.


In [10]:
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None
        
        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1
            
        vocab_size = len(counts)
        self.vocab_size = vocab_size
        
        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]
            
        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)
        
    def get_negative_sample(self, target):
        batch_size = target.shape[0]
    
        negative_sample = np.random.choice(self.vocab_size, 
                                            size=(batch_size, self.sample_size), 
                                            replace=True, p=self.word_p)
            
        return negative_sample


#### 4.2.7 네거티브 샘플링 구현

![image](https://github.com/choibigo/Study/assets/38881179/6969fc72-a164-45d6-907b-af2067df0f51)

In [5]:
import sys
sys.path.append('..')
from common.layers import SigmoidWithLoss

class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size #몇개를 sampling 해서 loss 계산에 사용할 것인지
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size+1)] # sampling된 개수 + target
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size+1)]
        self.params = []
        self.grads = []

        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target) # negtaive_sample 얻기

        # 긍정(정답) 레이블 순전파
        score = self.embed_dot_layers[0].forward(h, target)  # 정답 레이블 forward
        correct_label = np.ones(batch_size, dtype=np.int32) # batch를 고려하여 정답 (1) 벡터 만들기
        loss = self.loss_layers[0].forward(score, correct_label) # 정답 layer loss 계산

        # 부정(오답) 레이블 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i] # sample 1개씩 가져오기
            score = self.embed_dot_layers[i+1].forward(h, negative_target)
            loss += self.loss_layers[i+1].forward(score, negative_label)

        return loss
    
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh


        

## 4.3 개선판 word2vec 학습

#### 4.3.1 CBOW 모델 구현
- Embedding 계층과 Negative Sampling Loss 계층 적용

In [8]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Embedding

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        self.in_layers = []
        for i in range(2*window_size):
            self.in_layers.append(Embedding(W_in))
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        layers = self.in_layers + [self.ns_loss] # 모든 가중치와 기울기를 배열에 모은다.
        self.params = []
        self.grads = []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        self.wrod_vecs = W_in # 단어의 분산 표현을 저장한다.

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i]) # 각 맥락 벡터를 각 layer에 통과 시킨다. h는 forward output으로 weight와 내적의 결과를 모두 더한다.
        h *= 1 / len(self.in_layers) # score의 평균을 낸다.
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers) # 입력 layer 수로 나눈다.
        for layer in self.in_layers:
            layer.backward(dout) # 나눠진 미분 값을 각 layer에 backward 시킨다.
        return None

In [ ]:
import sys
sys.path.append('..')
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.utill import create_contexts_target
from data_set import ptb

# Hyper parameter
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# Load data
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)

# Model
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)``
trainer.plot()

word_vecs = model.wrod_vecs
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)



#### 4.3.3 CBOW 모델 평가

In [20]:
import sys
sys.path.append('..')
from common.utill import most_similar

import pickle

pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


- word2vec으로 얻은 단어의 분산 표현은 비슷한 단어를 가까이 모을 뿐 아니라, 더 복잡한 패턴을 파악하는 것으로 알려져 잇다.
- king - man + woman = queen 으로 유명한 문제가 있다.
- word2vec의 단어의 분산 표현을 사용하면 유추 문제를 벡터의 덧셈과 뺄셈으로 풀수 있다.
- 실제로 유추 문제를 풀려면 단어 벡터 공간에서 "man -> woman" 벡터와 "king -> ?" 벡터가 가능한 가까워지는 단어를 찾습니다.(woman-man = ? - king)
- 벡터의 연산을 통해서 이러한 문제를 풀 수 있다.
- woan-man+king 한 벡터를 normalize하고 분산 표현과 내적하여 similarity를 구한다.
- 이후 similarity와 가장 가까운 word를 구한다.

In [25]:
from common.utill import analogy

analogy('king', 'man', 'queen', word_to_id, id_to_word, word_vecs, top=5)
analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs, top=5)
analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs, top=5)
analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs, top=5)



[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875
